In [ ]:
# 이후 클래스 작성 시 함수의 파라메터와 리턴값은 클래스의 전역변수로 변환
def line(sdf=pb_df, df_dict=bdf_dict, is_batch=True):
    df = df_dict['LINE']
    print(f'최초 전선 데이터 크기: {df.shape}')
    
    # 숫자형 값 통일(실수형이 아닌 값을 실수형으로 변환)
    # (One-Hot Encoding시 동일한 컬럼값을 만들기 위해 실행)
    if df.LINE_SPEC_CD.dtype != 'float64':
        df['LINE_SPEC_CD'] = df['LINE_SPEC_CD'].astype(float)
    if df.NEUTRAL_SPEC_CD.dtype != 'float64':
        df['NEUTRAL_SPEC_CD'] = df['NEUTRAL_SPEC_CD'].astype(float)   
    # 중성선규격코드(NEUTRAL_SPEC_CD)에 0.0과 NaN이 존재(NaN=>999.0 변환)
    df['NEUTRAL_SPEC_CD'] = df['NEUTRAL_SPEC_CD'].fillna(999.0)
    # 중성선종류코드(NEUTRAL_TYPE_CD)의 NaN값을 문자열 'NaN'으로 치환
    df.NEUTRAL_TYPE_CD = df.NEUTRAL_TYPE_CD.fillna('NaN')
    # 결선방식이 41인 값이 1개만 존재하기 때문에 많이 있는 43으로 치환
    df.WIRING_SCHEME = df.WIRING_SCHEME.replace(41, 43)
    # 전선 전체길이 추가: = 선로길이(SPAN) * 전선 갯 수(PHASE)
    df.loc[:, 'LINE_LENGTH'] = df.SPAN * df.LINE_PHASE_CD

    # 결측치 처리
    df.fillna(0, inplace=True)
    
    # 코드형 컬럼 One-Hot Encoding
    # WIRING_SCHEME은 마지막에 '_CD'가 붙지 않음
    prefix = ['WIRING_SCHEME', 'LINE_TYPE', 'LINE_SPEC', 'LINE_PHASE',
              'NEUTRAL_TYPE', 'NEUTRAL_SPEC']
    columns = [x+'_CD' for x in prefix if x != 'WIRING_SCHEME']
    columns += ['WIRING_SCHEME']
    df = pd.get_dummies(df, columns=columns, prefix=prefix)
    # True, False를 1, 0으로 변환
    df = df.apply(lambda x: int(x) if isinstance(x, bool) else x)
    # 실시간 처리에서 동일 컬럼을 추가하기 위해 학습에서 나올 컬럼리스트 저장
    df_cols = df.columns.tolist()
    if is_batch:
        save_pickle(df_cols, file_code='DUMP,LINE_ONE_HOT_COLS')
    else:
        modeling_cols = read_pickle('DUMP,LINE_ONE_HOT_COLS')
        append_cols = [col for col in modeling_cols if col not in df_cols]
        df.loc[:, append_cols] = 0
    print(f'전선 One-Hot Encoding 결과: {df.shape}')
    
    # 공사비별 전선 데이터 합산
    unique_cons_ids = df.CONS_ID.unique()
    cons_id_line_sums = []
    sum_cols = ['SPAN'] + df.columns.tolist()[5:]
    for cid in unique_cons_ids:
        cons_id_line_sums.append(
            [cid]+df[df.CONS_ID==cid][sum_cols].sum().values.tolist())
    # 공사번호별로 합산된 전주 정보를 데이터프레임으로 변환
    line_sums_df = pd.DataFrame(
        cons_id_line_sums, columns=['CONS_ID']+sum_cols)
    
    # 공사비 데이터와 전주 그룹 데이터 병합
    pdf = pd.merge(
        sdf, line_sums_df,
        left_on='CONS_ID', right_on='CONS_ID', how='left')
    print(f'전선 그룹정보가 적용된 데이터 크기: {pdf.shape}')
    
    if is_batch:
        save_data(pdf, file_code='PP,LINE')
    return pdf